In [1]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors

In [2]:
data_tukang = pd.read_csv('data_tukang.csv')
data_user = pd.read_csv('data_user.csv')

In [3]:
tf_user = data_user.loc[:, ['Nama','Daerah','Order', 'Rating', 'Keterangan', 'Timestamp']]
tf_user = tf_user.rename(columns={'Nama': 'Nama User', 'Rating': 'User Rating'})
tf_user

,Nama User,Daerah,Order,User Rating,Keterangan,Timestamp
0,Andi Susanto,Purwokerto,Ningsih Instalasi Pipa,2.8,Pemasangan Pipa Sumur,20/12/2023
1,Andi Susanto,Purwokerto,Kurniawan Tukang Kayu,4.2,Servis Dipan Kasur,8/7/2023
2,Andi Susanto,Purwokerto,Lestari Tukang Bangunan dan Pengecetan,4.9,Cat Tembok,5/3/2023
3,Andi Susanto,Purwokerto,Lestari Instalasi Listrik,4.1,Service Kulkas,10/12/2023
4,Andi Susanto,Purwokerto,Sari Design dan Cetak,2.8,Design Pamflet dan Cetak,11/8/2023
...,...,...,...,...,...,...
155,Siti Wati,Semarang,Utami Tukang Aluminium,4.7,Pemasangan Plat Mesin,7/3/2023
156,Siti Wati,Semarang,Kusuma Tukang Kayu,2.8,Pembuatakn Lemari,21/8/2023
157,Siti Wati,Semarang,Susanto Perpipaan Gas,4.5,Perawatan Pipa,20/9/2023
158,Siti Wati,Semarang,Hartono Instalasi Pipa,4.9,Perbaikan Pipa,10/9/2023


In [4]:
tf_tukang = data_tukang.loc[:, ['Nama','Daerah','Tempat Pekerjaan','Keahlian', 'Harga', 'Rating', 'Pengrating']]
tf_tukang = tf_tukang.rename(columns={'Nama': 'Nama Tukang', 'Tempat Pekerjaan': 'Order','Rating': 'Rating Tukang', 'Pengrating': 'Jumlah Rating'})
tf_tukang

,Nama Tukang,Daerah,Order,Keahlian,Harga,Rating Tukang,Jumlah Rating
0,Ahmad Lestari,Sleman,Lestari Tukang Bangunan dan Pengecetan,Tukang Bangunan dan Pengecetan,95000,4.7,87
1,Wati Wibowo,Kebumen,Wibowo Tukang Aluminium,Tukang Aluminium,60000,4.7,80
2,Adi Sari,Semarang,Sari Tukang AC,Tukang AC,150000,4.8,20
3,Joko Saputra,Banyuwangi,Saputra Instalasi Pipa,Instalasi Pipa,70000,3.5,33
4,Sinta Gunawan,Wonosobo,Gunawan Tukang Listrik Rumah Tangga,Tukang Listrik Rumah Tangga,120000,3.5,1
...,...,...,...,...,...,...,...
995,Andi Susanto,Semarang,Susanto Design Logo,Design Logo,95000,4.8,14
996,Dewa Wijaya,Tasikmalaya,Wijaya Tukang Bangunan dan Pengecetan,Tukang Bangunan dan Pengecetan,70000,4.8,14
997,Rahayu Gunawan,Solo,Gunawan Tukang AC,Tukang AC,80000,4.3,14
998,Adi Hartono,Purwokerto,Hartono Tukang Listrik Rumah Tangga,Tukang Listrik Rumah Tangga,130000,4.8,46



 **K-Nearest Neighbors (KNN)** adalah algoritma pembelajaran mesin yang digunakan untuk masalah klasifikasi dan regresi. Algoritma ini bekerja berdasarkan prinsip bahwa data-data yang serupa cenderung berada dalam jarak yang dekat dalam ruang fitur. Ketika diberikan data baru, KNN mencari k-tetangga terdekat dari data tersebut di dalam ruang fitur (dengan k sebagai parameter yang ditentukan sebelumnya). Prediksi untuk data baru tersebut diambil berdasarkan mayoritas label k-tetangga terdekatnya dalam kasus klasifikasi, atau rerata nilai dalam kasus regresi.

Penggunaannya melibatkan beberapa langkah:

**Pemilihan Parameter K:** Parameter K harus ditentukan sebelumnya. Ini mengindikasikan jumlah tetangga terdekat yang akan diperhitungkan saat membuat prediksi. Pemilihan nilai K yang tepat bisa memengaruhi kinerja model.

**Pemrosesan Data:** Data perlu diproses agar cocok dengan model. Ini termasuk pengelompokkan data menjadi variabel independen (fitur) dan variabel dependen (label atau target).

**Perhitungan Jarak:** KNN mengukur jarak antara data baru dan data yang ada dalam ruang fitur. Pemilihan metrik jarak (misalnya Euclidean, Manhattan, Cosine) dapat berpengaruh pada hasil akhir.

**Penentuan Tetangga Terdekat:** Setelah data baru diberikan, KNN menghitung jarak antara data tersebut dengan semua titik data yang ada. Kemudian, menemukan K titik terdekat yang paling mirip.

**Prediksi:** Untuk masalah klasifikasi, label atau kategori dari mayoritas tetangga terdekat dipilih sebagai prediksi untuk data baru. Untuk regresi, rerata nilai dari tetangga terdekat digunakan sebagai prediksi.

In [5]:
def cari_tukang(nama_user, kebutuhan, tf_user, tf_tukang):
    # Mengambil daerah dari data berdasarkan nama pengguna
    lokasi_user = tf_user.loc[tf_user['Nama User'] == nama_user, 'Daerah'].values[0]

    # Menggunakan pemrosesan teks sederhana untuk pencocokan kebutuhan pengguna
    matching_jobs = tf_tukang[tf_tukang['Keahlian'].str.contains(kebutuhan, case=False)]

    # Memfilter berdasarkan lokasi (daerah dari data pengguna)
    matching_jobs = matching_jobs[matching_jobs['Daerah'] == lokasi_user]

    # Fitur yang akan digunakan untuk peringkat
    features = matching_jobs[['Rating Tukang', 'Jumlah Rating','Harga']]

    # Inisialisasi model KNN
    knn = NearestNeighbors(n_neighbors=len(matching_jobs))  # Jumlah tetangga = jumlah tukang yang cocok

    # Latih model dengan fitur yang dipilih
    knn.fit(features)

    # Mendapatkan indeks tukang yang cocok
    _, indices = knn.kneighbors(features)

    # Menampilkan hasil berdasarkan peringkat dari model
    hasil_terurut = matching_jobs.iloc[indices[0]]  # Menggunakan indeks dari model KNN

    # Mengurutkan hasil berdasarkan Rating Tukang secara descending
    hasil_terurut = hasil_terurut.sort_values(by='Rating Tukang', ascending=False)

    return hasil_terurut


Dalam fungsi **cari_tukang** yang diberikan, **algoritma K-Nearest Neighbors (KNN)** digunakan untuk merekomendasikan tukang yang paling sesuai dengan kebutuhan pengguna.

Proses dimulai dengan menentukan lokasi pengguna dari data yang disediakan. Selanjutnya, menggunakan informasi keahlian yang diperlukan oleh pengguna, fungsi tersebut memfilter tukang-tukang yang memiliki keahlian sesuai dengan kebutuhan tersebut. Setelah itu, fitur-fitur yang relevan seperti **'Rating Tukang', 'Jumlah Rating', dan 'Harga'** dari tukang-tukang yang cocok dipilih untuk melakukan peringkat.

Dengan menggunakan algoritma KNN, fungsi tersebut dapat memberikan rekomendasi tukang yang sesuai dengan preferensi dan lokasi pengguna berdasarkan data yang ada.




In [6]:
# Input dari pengguna
nama_user = input("Masukkan nama Anda: ")
kebutuhan_user = input("Masukkan jenis pekerjaan yang dibutuhkan: ")

# Memanggil fungsi untuk mencari tukang
hasil_pencarian = cari_tukang(nama_user, kebutuhan_user, tf_user, tf_tukang)

# Menampilkan hasil tukang yang paling sesuai dengan urutan rating descending
print("Berikut adalah tukang yang paling sesuai dengan kebutuhan Anda di", hasil_pencarian['Daerah'].iloc[0], ":")
hasil_pencarian[['Nama Tukang', 'Daerah', 'Order', 'Keahlian', 'Harga', 'Rating Tukang', 'Jumlah Rating']]

Masukkan nama Anda: Siti Wati
Masukkan jenis pekerjaan yang dibutuhkan: Listrik
Berikut adalah tukang yang paling sesuai dengan kebutuhan Anda di Semarang :


,Nama Tukang,Daerah,Order,Keahlian,Harga,Rating Tukang,Jumlah Rating
398,Citra Nugroho,Semarang,Nugroho Instalasi Listrik,Instalasi Listrik,150000,4.9,17
58,Hadi Susanto,Semarang,Susanto Instalasi Listrik,Instalasi Listrik,130000,4.8,5
17,Adi Saputri,Semarang,Saputri Tukang Listrik Rumah Tangga,Tukang Listrik Rumah Tangga,120000,4.7,80
362,Maya Hadi,Semarang,Hadi Tukang Listrik Rumah Tangga,Tukang Listrik Rumah Tangga,150000,4.6,80
667,Arief Pratama,Semarang,Pratama Instalasi Listrik,Instalasi Listrik,150000,4.6,88
276,Linda Nugroho,Semarang,Nugroho Tukang Listrik Rumah Tangga,Tukang Listrik Rumah Tangga,140000,4.3,20
619,Hendra Saputri,Semarang,Saputri Tukang Listrik Rumah Tangga,Tukang Listrik Rumah Tangga,130000,4.1,20
318,Surya Setiawan,Semarang,Setiawan Instalasi Listrik,Instalasi Listrik,100000,4.0,76
528,Rahayu Wibowo,Semarang,Wibowo Tukang Listrik Rumah Tangga,Tukang Listrik Rumah Tangga,90000,2.8,2
